In [1]:
import numpy as np
import pandas as pd

In [3]:
books = pd.read_csv('results/Books.csv')
ratings = pd.read_csv('results/Ratings.csv')

C:\Users\chill\AppData\Local\Temp\ipykernel_20864\3061105458.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('results/Books.csv')


In [4]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
print(books.shape)
print(ratings.shape)


(271360, 8)
(1149780, 3)


In [8]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [9]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [10]:
books.duplicated().sum()

0

In [11]:
ratings.duplicated().sum()

0

In [26]:
import mysql.connector
from mysql.connector import Error

def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host="127.0.0.1",    # As shown in your image
            port=3306,           # Port from your image
            user="root",         # Username from your image
            password="123456789"         # You'll need to enter your password
        )
        if connection.is_connected():
            print("Successfully connected to MySQL server")
            return connection
    except Error as e:
        print(f"Error connecting to MySQL server: {e}")
        return None

# Establish connection
connection = connect_to_mysql()
if connection:
    cursor = connection.cursor()
    print("Cursor created successfully")

Successfully connected to MySQL server
Cursor created successfully


In [31]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

def setup_database():
    try:
        connection = mysql.connector.connect(
            host="127.0.0.1",
            port=3306,
            user="root",
            password="123456789"
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            
            # Create database
            cursor.execute("DROP DATABASE IF EXISTS book_recommendation")
            cursor.execute("CREATE DATABASE book_recommendation")
            cursor.execute("USE book_recommendation")
            
            # Create books table with modified VARCHAR lengths
            cursor.execute("""
                CREATE TABLE books (
                    ISBN VARCHAR(20) PRIMARY KEY,
                    Book_Title VARCHAR(1000),
                    Book_Author VARCHAR(500),
                    Year_Of_Publication VARCHAR(20),
                    Publisher VARCHAR(500),
                    Image_URL_S VARCHAR(500),
                    Image_URL_M VARCHAR(500),
                    Image_URL_L VARCHAR(500)
                ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci
            """)
            
            # Create ratings table without foreign key constraint initially
            cursor.execute("""
                CREATE TABLE ratings (
                    User_ID INT,
                    ISBN VARCHAR(20),
                    Book_Rating INT,
                    PRIMARY KEY (User_ID, ISBN)
                ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci
            """)
            
            print("Database and tables created successfully")
            return connection
            
    except Error as e:
        print(f"Error: {e}")
        return None

def clean_data(value):
    if pd.isna(value):
        return None
    return str(value).strip()

def batch_insert_data(connection, batch_size=1000):
    try:
        cursor = connection.cursor()
        
        # First, clean and prepare books data
        print("Preparing books data...")
        books_data = []
        for _, row in books.iterrows():
            try:
                books_data.append((
                    clean_data(row['ISBN']),
                    clean_data(row['Book-Title']),
                    clean_data(row['Book-Author']),
                    clean_data(row['Year-Of-Publication']),
                    clean_data(row['Publisher']),
                    clean_data(row['Image-URL-S']),
                    clean_data(row['Image-URL-M']),
                    clean_data(row['Image-URL-L'])
                ))
            except Exception as e:
                print(f"Error processing book row: {row['ISBN']}, Error: {e}")
                continue

        # Insert books data in batches
        print(f"Inserting {len(books_data)} books...")
        for i in range(0, len(books_data), batch_size):
            batch = books_data[i:i + batch_size]
            try:
                cursor.executemany("""
                    INSERT IGNORE INTO books 
                    (ISBN, Book_Title, Book_Author, Year_Of_Publication,
                     Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, batch)
                connection.commit()
                print(f"Inserted books {i} to {i + len(batch)}")
            except Error as e:
                print(f"Error inserting books batch {i}: {e}")
                connection.rollback()

        # Get list of valid ISBNs from books table
        cursor.execute("SELECT ISBN FROM books")
        valid_isbns = {row[0] for row in cursor.fetchall()}

        # Prepare ratings data (only for books that exist)
        print("Preparing ratings data...")
        ratings_data = []
        for _, row in ratings.iterrows():
            try:
                isbn = clean_data(row['ISBN'])
                if isbn in valid_isbns:  # Only include ratings for existing books
                    ratings_data.append((
                        int(row['User-ID']),
                        isbn,
                        int(row['Book-Rating'])
                    ))
            except Exception as e:
                print(f"Error processing rating row: {row['ISBN']}, Error: {e}")
                continue

        # Insert ratings data in batches
        print(f"Inserting {len(ratings_data)} ratings...")
        for i in range(0, len(ratings_data), batch_size):
            batch = ratings_data[i:i + batch_size]
            try:
                cursor.executemany("""
                    INSERT IGNORE INTO ratings (User_ID, ISBN, Book_Rating)
                    VALUES (%s, %s, %s)
                """, batch)
                connection.commit()
                print(f"Inserted ratings {i} to {i + len(batch)}")
            except Error as e:
                print(f"Error inserting ratings batch {i}: {e}")
                connection.rollback()

        # Add foreign key constraint after data insertion
        try:
            cursor.execute("""
                ALTER TABLE ratings
                ADD CONSTRAINT fk_ratings_books
                FOREIGN KEY (ISBN) REFERENCES books(ISBN)
            """)
            print("Foreign key constraint added successfully")
        except Error as e:
            print(f"Error adding foreign key constraint: {e}")

        # Verify data insertion
        cursor.execute("SELECT COUNT(*) FROM books")
        books_count = cursor.fetchone()[0]
        cursor.execute("SELECT COUNT(*) FROM ratings")
        ratings_count = cursor.fetchone()[0]
        print(f"\nInsertion Summary:")
        print(f"Total books inserted: {books_count}")
        print(f"Total ratings inserted: {ratings_count}")
        
    except Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

# Main execution
try:
    print("Starting database setup and data insertion...")
    connection = setup_database()
    if connection:
        batch_insert_data(connection)
        print("Data insertion completed successfully")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'connection' in locals() and connection and connection.is_connected():
        connection.close()
        print("Database connection closed")

Starting database setup and data insertion...
Database and tables created successfully
Preparing books data...
Inserting 271360 books...
Inserted books 0 to 1000
Inserted books 1000 to 2000
Inserted books 2000 to 3000
Inserted books 3000 to 4000
Inserted books 4000 to 5000
Inserted books 5000 to 6000
Inserted books 6000 to 7000
Inserted books 7000 to 8000
Inserted books 8000 to 9000
Inserted books 9000 to 10000
Inserted books 10000 to 11000
Inserted books 11000 to 12000
Inserted books 12000 to 13000
Inserted books 13000 to 14000
Inserted books 14000 to 15000
Inserted books 15000 to 16000
Inserted books 16000 to 17000
Inserted books 17000 to 18000
Inserted books 18000 to 19000
Inserted books 19000 to 20000
Inserted books 20000 to 21000
Inserted books 21000 to 22000
Inserted books 22000 to 23000
Inserted books 23000 to 24000
Inserted books 24000 to 25000
Inserted books 25000 to 26000
Inserted books 26000 to 27000
Inserted books 27000 to 28000
Inserted books 28000 to 29000
Inserted books 

## Popularity Based Recommender System

In [12]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [13]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [14]:
print(ratings_with_name.dtypes)


User-ID                 int64
ISBN                   object
Book-Rating             int64
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object


In [15]:
print(ratings_with_name['Book-Rating'].unique())


[ 0  5  9  8  6  7  4 10  3  2  1]


In [16]:
ratings_with_name['Book-Rating'] = pd.to_numeric(ratings_with_name['Book-Rating'], errors='coerce')


In [17]:
ratings_with_name['Book-Rating'].fillna(0, inplace=True)


In [18]:
avg_rating_df = ratings_with_name.groupby('Book-Title', as_index=False)['Book-Rating'].mean()
avg_rating_df.rename(columns={'Book-Rating': 'avg_rating'}, inplace=True)
avg_rating_df.head()


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [20]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [21]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [22]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [23]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

## Collaborative Filtering Based Recommender System

In [24]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [25]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [26]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [27]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [28]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [29]:
pt.fillna(0,inplace=True)

In [30]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity_scores = cosine_similarity(pt)

In [33]:
similarity_scores.shape

(706, 706)

In [34]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [35]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg']]

In [36]:
pt.index[545]

"The Handmaid's Tale"

In [37]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [38]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [39]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))